In [11]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

In [12]:
base_folder = Path('./result/ud-pipe')

nilc_reference_complete = pd.read_csv(base_folder / 'reference-complete.csv')
nilc_reference_simplified = pd.read_csv(base_folder / "reference-simplified.csv")
nilc_generated = pd.read_csv(base_folder / "generated-simplified.csv")

In [13]:
nilc_generated

,id,name,year,model,generated_with,non_svo_ratio,passive_voice_ratio,words_before_main_verb_mean,personal_pronoun_ratio,coreference_pronoun_ratio,demonstrative_pronoun_ratio,long_sentence_ratio,foreign_word_ratio
0,1,2022_ufc_inova_1_stripped.predicted,2022,portuguese-porttinari-ud-2.15-241121,mistralai/ministral-8b,0.024523,0.103542,2.688235,0.000406,0.000000,0.250000,0.209809,0.010764
1,1,2023_ufc_inova_1_stripped.predicted,2023,portuguese-porttinari-ud-2.15-241121,mistralai/ministral-8b,0.026549,0.132743,5.340659,0.000279,0.000000,0.800000,0.247788,0.005384
2,3,2021_ufc_inova_3_stripped.predicted,2021,portuguese-porttinari-ud-2.15-241121,mistralai/ministral-8b,0.056769,0.248908,5.160305,0.001669,0.000000,1.500000,0.310044,0.004662
3,1,2021_ufc_inova_1_stripped.predicted,2021,portuguese-porttinari-ud-2.15-241121,mistralai/ministral-8b,0.017857,0.148810,8.133065,0.000637,2.000000,1.384615,0.172619,0.007843
4,4,2025_ufc_inova_4_stripped.predicted,2025,portuguese-porttinari-ud-2.15-241121,mistralai/ministral-8b,0.023715,0.134387,4.692308,0.000558,0.000000,1.333333,0.181818,0.008152
5,1,2025_ufc_inova_1_stripped.predicted,2025,portuguese-porttinari-ud-2.15-241121,mistralai/ministral-8b,0.011628,0.348837,6.869565,0.002499,0.000000,2.260870,0.418605,0.008065
6,2,2021_ufc_inova_2_stripped.predicted,2021,portuguese-porttinari-ud-2.15-241121,mistralai/ministral-8b,0.076923,0.179487,15.854167,0.002051,0.000000,0.714286,0.166667,0.000000
7,1,2022_ufc_inova_1_stripped.predicted,2022,portuguese-porttinari-ud-2.15-241121,qwen/qwen3-14b,0.027778,0.076389,3.461538,0.001983,2.000000,1.200000,0.099537,0.013333
8,1,2023_ufc_inova_1_stripped.predicted,2023,portuguese-porttinari-ud-2.15-241121,qwen/qwen3-14b,0.027972,0.080420,5.260563,0.000756,0.000000,1.500000,0.115385,0.011161
9,3,2021_ufc_inova_3_stripped.predicted,2021,portuguese-porttinari-ud-2.15-241121,qwen/qwen3-14b,0.016575,0.104972,4.767123,0.001213,0.000000,0.714286,0.154696,0.008658


In [14]:
proportional_metrics = [
 'personal_pronoun_ratio', # can also be a positive sign according to Unicamp,
]

inverse_proportional_metrics = [

    "coreference_pronoun_ratio",
    'demonstrative_pronoun_ratio',
    'non_svo_ratio',
    'passive_voice_ratio',
    'words_before_main_verb_mean', # use different scaling for this metric?
    "long_sentence_ratio",
    "foreign_word_ratio",
]

complete_metrics = proportional_metrics + inverse_proportional_metrics

### Teste T

In [15]:
import numpy as np
import scipy.stats as st
import csv
from typing import Any
import math

confidence = 0.95

def generate_confidence_interval(df:pd.DataFrame,metrics:list[str])->pd.DataFrame:
    confidence_intervals: list[dict[str,Any]] = []
    for metric in metrics:
        per_group = df.groupby("generated_with",dropna=False)[f"{metric}"]
        for row in per_group.agg(["mean","count",st.sem]).iterrows():
            range_min,range_max = st.t.interval(confidence=confidence,df=row[1]["count"]-1,loc=row[1]["mean"],scale=row[1]["sem"])
            confidence_intervals.append({"metric":metric,"model":row[0], "range_min":range_min,"mean": row[1]["mean"], "range_max":range_max})

    return pd.DataFrame(confidence_intervals)

generate_confidence_interval(nilc_reference_complete,complete_metrics).to_csv('nilc-reference-complete-confidence-intervals.csv')

generate_confidence_interval(nilc_reference_simplified,complete_metrics).to_csv('nilc-reference-simplified-confidence-intervals.csv')

generate_confidence_interval(nilc_generated,complete_metrics).to_csv('nilc-generated-simplified-confidence-intervals.csv')

/home/joao/projects/text-simplification/.venv/lib/python3.12/site-packages/scipy/stats/_distn_infrastructure.py:2304: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/joao/projects/text-simplification/.venv/lib/python3.12/site-packages/scipy/stats/_distn_infrastructure.py:2305: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/home/joao/projects/text-simplification/.venv/lib/python3.12/site-packages/scipy/stats/_distn_infrastructure.py:2304: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/joao/projects/text-simplification/.venv/lib/python3.12/site-packages/scipy/stats/_distn_infrastructure.py:2305: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
